In [36]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import time
import datetime
from sklearn import preprocessing, cross_validation, svm, metrics
from sklearn.cluster import KMeans

df = pd.read_csv("data/train.csv")
df.head()

,ln,dob,gn,fn
0,SMITH JR,01/03/68,F,WILLIAM
1,ROTHMEYER JR,01/03/68,F,WILLIAM
2,ASBY JR,01/03/68,F,WILLIAM
3,SALTER JR,01/03/68,F,WILLIAM
4,SALTER JR,01/03/68,F,WILLIAM


In [37]:
df['fn_len'] = df['fn'].apply(lambda val:len(val))

In [38]:
df.head(50)

,ln,dob,gn,fn,fn_len
0,SMITH JR,01/03/68,F,WILLIAM,7
1,ROTHMEYER JR,01/03/68,F,WILLIAM,7
2,ASBY JR,01/03/68,F,WILLIAM,7
3,SALTER JR,01/03/68,F,WILLIAM,7
4,SALTER JR,01/03/68,F,WILLIAM,7
5,BLAND JR,21/02/62,F,WILLIAM,7
6,BLAND JR,21/02/62,F,WILLIAM,7
7,BLAND JR,21/02/62,F,WILLIAM,7
8,SHAFFER JR,21/02/62,F,WILLIAM,7
9,SHAFFER JR,21/02/62,F,WILLIAM,7


In [39]:
def encode(inp):
    j = 0
    data = pd.DataFrame()
    data['index']   = inp
    data['label'] = inp
    data['label'][0] = 0
    for i in range(1,len(data)):
        if(data['index'][i] in data['index'][i-1]):
            data['label'][i] = data['label'][i-1]
        else:
            j += 1
            data['label'][i] = j
    return data['label']

In [40]:
df.sort_values(['fn'],ascending=False,inplace=True)
df.reset_index(inplace=True)
df.drop(['index'],1,inplace=True)
df['fn_label'] = encode(df['fn'])
df.head(5)

,ln,dob,gn,fn,fn_len,fn_label
0,SMITH JR,01/03/68,F,WILLIAM,7,0
1,BLAND III,21/02/62,F,WILLIAM,7,0
2,SHAFFER JR,25/10/53,F,WILLIAM,7,0
3,BLAND JR,25/10/53,F,WILLIAM,7,0
4,BLAND JR,25/10/53,F,WILLIAM,7,0


In [41]:
df.sort_values(['ln'],ascending=False,inplace=True)
df.reset_index(inplace=True)
df.drop(['index'],1,inplace=True)
df['ln_label'] = encode(df['ln'])
df.head(50)

,ln,dob,gn,fn,fn_len,fn_label,ln_label
0,SMITH JR,01/03/68,F,WILLIAM,7,0,0
1,SMITH JR,07/10/37,M,HAROLD,6,13,0
2,SHAFFER JR,25/10/53,F,WILLIAM,7,0,1
3,SHAFFER JR,21/02/62,F,WILLIAM,7,0,1
4,SHAFFER JR,21/02/62,F,WILLIAM,7,0,1
5,SALTER JR,01/03/68,F,WILLIAM,7,0,2
6,SALTER JR,01/03/68,F,WILLIAM,7,0,2
7,ROTHMEYER JR,01/03/68,F,WILLIAM,7,0,3
8,MUSTAPHA JR,07/10/37,M,HAROLD,6,13,4
9,MUSTAPHA JR,11/03/47,M,OWEN,4,5,4


In [42]:
df["val_gn"] = df["gn"].map({"M":0,"F":1})

In [43]:
def DOB_to_unix(str):
    s = str.split('/')
    str = s[0] + '/' + s[1]+ '/19' + s[2]
    
    return int((time.mktime(datetime.datetime.strptime(str, "%d/%m/%Y").timetuple())))

df["val_dob"] = df["dob"].apply(lambda x: DOB_to_unix(x))

In [44]:
df.head()

,ln,dob,gn,fn,fn_len,fn_label,ln_label,val_gn,val_dob
0,SMITH JR,01/03/68,F,WILLIAM,7,0,0,1,-57994200
1,SMITH JR,07/10/37,M,HAROLD,6,13,0,0,-1017293400
2,SHAFFER JR,25/10/53,F,WILLIAM,7,0,1,1,-510816600
3,SHAFFER JR,21/02/62,F,WILLIAM,7,0,1,1,-248074200
4,SHAFFER JR,21/02/62,F,WILLIAM,7,0,1,1,-248074200


In [45]:
train = np.array(df[['fn_label','ln_label','val_gn','val_dob']])
scaler = preprocessing.StandardScaler().fit(train)
train = scaler.transform(train)
train[:50]

array([[-1.3825181 , -1.67527297,  1.7209121 ,  2.39653414],
       [ 0.60161023, -1.67527297, -0.5810872 , -0.82671534],
       [-1.3825181 , -1.50774567,  1.7209121 ,  0.87504891],
       [-1.3825181 , -1.50774567,  1.7209121 ,  1.75786452],
       [-1.3825181 , -1.50774567,  1.7209121 ,  1.75786452],
       [-1.3825181 , -1.34021837,  1.7209121 ,  2.39653414],
       [-1.3825181 , -1.34021837,  1.7209121 ,  2.39653414],
       [-1.3825181 , -1.17269108,  1.7209121 ,  2.39653414],
       [ 0.60161023, -1.00516378, -0.5810872 , -0.82671534],
       [-0.61939182, -1.00516378, -0.5810872 ,  0.17251232],
       [-0.46676656, -1.00516378, -0.5810872 ,  0.17251232],
       [ 1.66998703, -0.83763648, -0.5810872 ,  0.18093115],
       [-1.07726759, -0.83763648, -0.5810872 ,  0.87504891],
       [-0.00889079, -0.83763648, -0.5810872 , -0.06409784],
       [-0.00889079, -0.83763648, -0.5810872 , -0.64527511],
       [-0.00889079, -0.83763648, -0.5810872 , -0.55673227],
       [-0.00889079, -0.

In [55]:
n_cluster = len(train)
from sklearn.cluster import MiniBatchKMeans,AffinityPropagation
# kmn = KMeans(n_clusters=n_cluster)
kmn = MiniBatchKMeans()
kmn.fit(train)
groups = kmn.predict(train)
print(len(groups))
out = df
print(len(out))
out['type'] = groups
print(len(out['type'].unique()))

103
103
8


In [47]:
out.sort_values(['type'],ascending=False,inplace=True)
# out.drop_duplicates('type',inplace=True)
out.reset_index(inplace=True)
out.drop(['index'],1,inplace=True)
out.head()

,ln,dob,gn,fn,fn_len,fn_label,ln_label,val_gn,val_dob,type
0,MELVIN JR,07/10/37,M,HAROLD,6,13,6,0,-1017293400,55
1,MELVIN JR,07/10/37,M,HAROLD,6,13,6,0,-1017293400,55
2,MELVIN JR,07/10/37,M,HAROLD,6,13,6,0,-1017293400,55
3,MELVIN JR,07/10/37,M,HAROLD,6,13,6,0,-1017293400,55
4,MELVIN JR,07/10/37,M,HAROLD,6,13,6,0,-1017293400,55


In [48]:
out.reset_index(inplace=True)
out.drop(['index'],1,inplace=True)
out.head()
out = df[['ln','dob','gn','fn']]

In [14]:
out.to_csv('type.csv',index=None)